# VinBigData Chest X-ray Abnormalities Detection
Automatically localize and classify thoracic abnormalities from chest radiographs

#### The aim of this notebook is to explore the histogram equalization technique and it's variant.

## Import libraries

In [ ]:
!pip install tensorflow_io

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio

## Read DICOM images

In [ ]:
# Reading DICOM images

def read_dicom(path):
    image_bytes = tf.io.read_file(path)
    image = tfio.image.decode_dicom_image(
        image_bytes, 
        dtype = tf.uint16
    )
    
    image = tf.squeeze(image, axis = 0)
    
    image = tf.image.resize(
        image, 
        (500, 500), 
        preserve_aspect_ratio = True
    )
    
    image = image - tf.reduce_min(image)
    image = image / tf.reduce_max(image)
    image = tf.cast(image * 255, tf.uint8)
    
    return image

## Visualization

### Original

In [ ]:
fpath = "../input/vinbigdata-chest-xray-abnormalities-detection/train/000434271f63a053c4128a0ba6352c7f.dicom"
image = read_dicom(fpath)

plt.figure(figsize = (18, 4))

plt.subplot(1, 2, 1)
plt.axis("off")
plt.title("Original")
plt.imshow(image, cmap = "gray");

plt.subplot(1, 2, 2)
hist, bins = np.histogram(image.numpy().flatten(), 256, [0, 256])
cdf = hist.cumsum()
cdf_normalized = cdf * float(hist.max()) / cdf.max()
plt.plot(cdf_normalized, color = 'b')

plt.hist(image.numpy().flatten(), 256, [0, 256], color = 'r')
plt.xlim([0, 256])
plt.legend(('cdf', 'histogram'), loc = 'upper left')
plt.title("Original")
plt.show();

### Histogram Equalization
This method usually increases the global contrast of images. This is done by redistributing pixel intensity.

In [ ]:
plt.figure(figsize = (18, 4))

plt.subplot(1, 2, 1)
plt.axis("off")
plt.title("Histogram Equalization")

img_he = cv2.equalizeHist(image.numpy())

plt.imshow(img_he, cmap = "gray");

plt.subplot(1, 2, 2)
hist, bins = np.histogram(img_he.flatten(), 256, [0, 256])
cdf = hist.cumsum()
cdf_normalized = cdf * float(hist.max()) / cdf.max()
plt.plot(cdf_normalized, color = 'b')

plt.hist(img_he.flatten(), 256, [0, 256], color = 'r')
plt.xlim([0, 256])
plt.legend(('cdf', 'histogram'), loc = 'upper left')
plt.title("Histogram Equalization")
plt.show();

A disadvantage of the method is that it is indiscriminate. It may increase the contrast of background noise.

### Contrast Limited Adaptive Histogram Equalization
Technique to improve the contrast of an image. It differs from ordinary histogram equalization in the respect that the adaptive method computes several histograms, each corresponding to a distinct section of the image, and uses them to redistribute the lightness values of the image. Additionally, it prevents over amplification of noise.

In [ ]:
plt.figure(figsize = (18, 4))

plt.subplot(1, 2, 1)
plt.axis("off")
plt.title("Contrast Limited Adaptive Histogram Equalization")

clahe = cv2.createCLAHE(
    clipLimit = 2., 
    tileGridSize = (10, 10)
)
    
img_clahe = clahe.apply(image.numpy()) 

plt.imshow(img_clahe, cmap = "gray");

plt.subplot(1, 2, 2)
hist, bins = np.histogram(img_clahe.flatten(), 256, [0, 256])
cdf = hist.cumsum()
cdf_normalized = cdf * float(hist.max()) / cdf.max()
plt.plot(cdf_normalized, color = 'b')

plt.hist(img_clahe.flatten(), 256, [0, 256], color = 'r')
plt.xlim([0, 256])
plt.legend(('cdf', 'histogram'), loc = 'upper left')
plt.title("Contrast Limited Adaptive Histogram Equalization")
plt.show();

### Summary

In [ ]:
plt.figure(figsize = (18, 4))

plt.subplot(1, 3, 1)
plt.axis("off")
plt.title("Original")
plt.imshow(image, cmap = "gray");

plt.subplot(1, 3, 2)
plt.axis("off")
plt.title("Histogram Equalization")
plt.imshow(img_he, cmap = "gray");

plt.subplot(1, 3, 3)
plt.axis("off")
plt.title("Contrast Limited Adaptive Histogram Equalization")
plt.imshow(img_clahe, cmap = "gray");